In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
import os

def load_sam():
    checkpoint_path = "../checkpoints/sam_vit_h_4b8939.pth"
    model_type = "vit_h"
    
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"SAM checkpoint not found at {checkpoint_path}")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Initialize the SAM model with the specified type
    sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
    
    sam.to(device=device)
    
    return sam, device

class InteractiveSAM:
    def __init__(self, sam, device):
        self.predictor = SamPredictor(sam)
        self.device = device
        self.points = []
        self.point_labels = []
        self.current_mask = None
        self.box_start = None
        self.box_end = None
        self.drawing_box = False
        self.box_mode = False  # Box mode toggle
        self.image = None
        self.box_rect = None
        
    def set_image(self, image_path):
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at: {image_path}")
                
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image at: {image_path}")
                
        self.image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.predictor.set_image(self.image)
            
        # Get image dimensions
        self.height, self.width = self.image.shape[:2]
            
    def generate_bounding_box_from_mask(self):
        # Ensure a mask has been generated
        if self.current_mask is None:
            raise ValueError("No segmentation mask available to compute bounding box.")
        
        # Find where the mask is True
        y_indices, x_indices = np.where(self.current_mask)
        
        if len(x_indices) == 0 or len(y_indices) == 0:
            raise ValueError("Segmentation mask is empty. Cannot compute bounding box.")
        
        # Compute tight bounding box coordinates
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()
        
        bounding_box = [x_min, y_min, x_max, y_max]
        return bounding_box
    
    def interactive_bounding_box(self):
        if not hasattr(self, 'image'):
            raise ValueError("No image has been set. Please call set_image() first.")
                
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        ax1.imshow(self.image)
        ax2.imshow(self.image)
        ax1.set_title("Controls:\nLeft/Right click: Add points\nMiddle click: Generate bounding box\nPress 'b' to toggle box mode")
        ax2.set_title("Bounding Box Result")
            
        def on_key_press(event):
            if event.key == 'b':
                self.box_mode = not self.box_mode
                mode = "ON" if self.box_mode else "OFF"
                print(f"Box mode toggled {mode}")
        
        def on_button_press(event):
            if event.inaxes == ax1:
                if self.box_mode and event.button == 1:  # Left click starts box
                    self.drawing_box = True
                    self.box_start = [event.xdata, event.ydata]
                elif not self.box_mode:
                    if event.button in [1, 3]:  # Left or right click for points
                        self.points.append([event.xdata, event.ydata])
                        self.point_labels.append(1 if event.button == 1 else 0)
                            
                        color = 'go' if event.button == 1 else 'ro'
                        ax1.plot(event.xdata, event.ydata, color, markersize=10)
                        plt.draw()
                    elif event.button == 2:  # Middle click to generate bounding box
                        self.generate_mask()
                        bounding_box = self.generate_bounding_box_from_mask()
                        self.display_bounding_box(ax2, bounding_box)
                        plt.draw()
                    
        def on_button_release(event):
            if self.drawing_box and event.inaxes == ax1:
                self.drawing_box = False
                self.box_end = [event.xdata, event.ydata]
                self.draw_box(ax1)
                self.generate_mask_with_box()
                bounding_box = self.generate_bounding_box_from_mask()
                self.display_bounding_box(ax2, bounding_box)
                plt.draw()
                
        def on_motion(event):
            if self.drawing_box and event.inaxes == ax1:
                if self.box_rect:
                    self.box_rect.remove()
                x0, y0 = self.box_start
                x1, y1 = event.xdata, event.ydata
                self.box_rect = plt.Rectangle((min(x0, x1), min(y0, y1)), 
                                              abs(x1 - x0), abs(y1 - y0),
                                              fill=False, color='yellow', linewidth=2)
                ax1.add_patch(self.box_rect)
                plt.draw()
                
        fig.canvas.mpl_connect('key_press_event', on_key_press)
        fig.canvas.mpl_connect('button_press_event', on_button_press)
        fig.canvas.mpl_connect('button_release_event', on_button_release)
        fig.canvas.mpl_connect('motion_notify_event', on_motion)
        plt.show()
            
        return self.current_mask
        
    def draw_box(self, ax):
        if self.box_start and self.box_end:
            x0, y0 = self.box_start
            x1, y1 = self.box_end
            if self.box_rect:
                self.box_rect.remove()
            self.box_rect = plt.Rectangle((min(x0, x1), min(y0, y1)), 
                                          abs(x1 - x0), abs(y1 - y0),
                                          fill=False, color='yellow', linewidth=2)
            ax.add_patch(self.box_rect)
            plt.draw()
    
    def generate_mask(self):
        if len(self.points) > 0:
            input_points = np.array(self.points)
            input_labels = np.array(self.point_labels)
                
            masks, scores, logits = self.predictor.predict(
                point_coords=input_points,
                point_labels=input_labels,
                multimask_output=False  # Single mask output
            )
                
            self.current_mask = masks[0]
        else:
            raise ValueError("No points provided for segmentation.")
    
    def generate_mask_with_box(self):
        if self.box_start and self.box_end:
            x0, y0 = self.box_start
            x1, y1 = self.box_end
            input_box = np.array([min(x0, x1), min(y0, y1), max(x0, x1), max(y0, y1)])
                
            masks, scores, logits = self.predictor.predict(
                point_coords=np.array(self.points) if len(self.points) > 0 else None,
                point_labels=np.array(self.point_labels) if len(self.points) > 0 else None,
                box=input_box[None, :],
                multimask_output=False  # Single mask output
            )
                
            self.current_mask = masks[0]
        else:
            raise ValueError("Bounding box coordinates are not defined.")
    
    def display_bounding_box(self, ax, bounding_box):
        x_min, y_min, x_max, y_max = bounding_box
        ax.clear()
        ax.imshow(self.image)
        ax.add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, 
                                   fill=False, edgecolor='red', linewidth=2))
        ax.set_title("Bounding Box Result")
        plt.draw()

def main():
    try:
        # Load model
        print("Loading SAM model...")
        sam, device = load_sam()
        
        # Create interactive segmentation instance
        interactive_sam = InteractiveSAM(sam, device)
        
        # Set image path
        image_path = "../data/raw/test/n01496331_3.jpg"
        
        try:
            interactive_sam.set_image(image_path)
        except (FileNotFoundError, ValueError) as e:
            print(f"Error: {e}")
            return
        
        # Start interactive bounding box creation
        print("\nInstructions:")
        print("- Left click: Add points for the object you want to segment")
        print("- Right click: Add points for the background")
        print("- Middle click: Generate the bounding box")
        print("- Press 'b' to toggle box drawing mode")
        print("- In box mode, left click and drag to draw a bounding box")
        print("- Close the window when done")
        
        mask = interactive_sam.interactive_bounding_box()
        
        # Display results
        if mask is not None:
            bounding_box = interactive_sam.generate_bounding_box_from_mask()
            x_min, y_min, x_max, y_max = bounding_box
            
            print(f"Bounding box coordinates: {bounding_box}")
            
            plt.figure(figsize=(15, 5))
            
            plt.subplot(131)
            plt.imshow(interactive_sam.image)
            plt.title("Original Image")
            
            plt.subplot(132)
            plt.imshow(interactive_sam.image)
            plt.imshow(mask, alpha=0.5, cmap='cool')
            plt.contour(mask, colors='red', linewidths=2)
            plt.title("Segmentation Mask")
            
            plt.subplot(133)
            plt.imshow(interactive_sam.image)
            plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, 
                                        fill=False, edgecolor='yellow', linewidth=2))
            plt.title("Bounding Box")
            
            plt.show()
                    
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


Loading SAM model...
Using device: cpu


C:\Users\2012b\PycharmProjects\underwater\.venv\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)